# Use examples of [premise](https://github.com/romainsacchi/premise) with user-generated scenarios

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `premise` to adapt the life cycle inventory database [ecoinvent](https://www.ecoinvent.org/) for prospective environmental impact assessment, using **user-generated scenarios**.

User-generated scenario are scenarios built by the premise users community. They can be used on their own
or together with a global IAM scenario.
Public community scenarios are listed under the repository [Premise community scenario](https://github.com/premise-community-scenarios).

User-generated scenarios are "packaged" into [data packages](https://specs.frictionlessdata.io/data-package/).

Data packages ensure that the data used comes with all necessary metadata, resources (scenario data, inventories, etc.), and that the data is formatted in a correct way.

To fetch a data packge, you can use the `datapackage` library.
For exmaple, let's fetch the user-generated scenario about [Switzerland's future energy supply](https://github.com/premise-community-scenarios/energy-perspective-2050-switzerland).

In [1]:
from datapackage import Package

In [2]:
# URL pointing to the raw datapackage.json file of the scenario
url = "https://raw.githubusercontent.com/premise-community-scenarios/energy-perspective-2050-switzerland/main/datapackage.json"
switzerland_2050 = Package(url)

The datapackage has 3 resources: scenario data, inventories and a configuration file.

In [12]:
for resource in switzerland_2050.resources:
    print(resource.name)

scenario_data
inventories
config


or

In [21]:
switzerland_2050.resource_names

['scenario_data', 'inventories', 'config']

And you can directly read them (or look the resources up directly from the repo):

In [27]:
import pandas as pd
data = switzerland_2050.get_resource("scenario_data").read()
pd.DataFrame(data).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,image,SSP2-Base,Business As Usual,CH,Production|Electricity|Run-of-river hydro,PJ/year,64.002,63.994,63.928,63.119,61.774,59.235,58.097,58.097,58.097,58.097,58.097,58.097
1,image,SSP2-Base,Business As Usual,CH,Production|Electricity|Reservoir,PJ/year,76.367,79.679,81.45,79.47,79.556,81.238,82.102,82.102,82.102,82.102,82.102,82.102
2,image,SSP2-Base,Business As Usual,CH,Production|Electricity|Nuclear|Boiling water r...,PJ/year,37.30202,28.321,15.165,0,0,0,0,0,0,0,0,0
3,image,SSP2-Base,Business As Usual,CH,Production|Electricity|Nuclear|Pressure water ...,PJ/year,42.06398,31.936,17.102,0,0,0,0,0,0,0,0,0
4,image,SSP2-Base,Business As Usual,CH,Production|Electricity|Conventional|Waste-to-E...,PJ/year,6.767,6.883,6.998,7.114,7.229,7.345,7.461,7.461,7.461,7.461,7.461,7.461


The datapackage can also be locally stored (i.e., on your computer), in which case you only
need to provide the filepath to the `datapackage.json` file.

Once the datapackage is loaded, you just need to pass it (in a list) to the `NewDatabase` instance of `premise`.

In [ ]:
from premise import *
import bw2data
bw2data.projects.set_current("your project name")

In [ ]:
scenarios = [
    {"model": "image", "pathway":"SSP2-Base", "year": 2040},
]

ndb = NewDatabase(
        scenarios = scenarios,        
        source_db="ecoinvent 3.8 cutoff",
        source_version="3.8",
        key='xxxxx',
        external_scenarios=[
            switzerland_2050,
        ]
)


To integrate the projections of the user-generated scenario, call `update_external_scenario()`.

In [ ]:
ndb.update_external_scenario()

You can combine the user-generated scenario with any, all or none of the projections for the IAM model.
For example, here with the electricity projections of the IMAGE SSP2-Base scenario:

In [ ]:
scenarios = [
    {"model": "image", "pathway":"SSP2-Base", "year": 2040},
]

ndb = NewDatabase(
        scenarios = scenarios,        
        source_db="ecoinvent 3.8 cutoff",
        source_version="3.8",
        key='xxxxx',
        external_scenarios=[
            switzerland_2050,
        ]
)

ndb.update_electricity()
ndb.update_external_scenario()

Once the projections are integrated, you can export the database(s) back to your Brightway2 project, to
a CSV Simapro file, or as a set of sparse amtrices (see main exmaples notebook for more details).